In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import psycopg2
import time
from time import localtime, strftime

In [ ]:
conn = psycopg2.connect(database="?", user="?", password="?", host="?", port="?")
conn.autocommit = True
cur = conn.cursor()

In [ ]:
#voucher price add
tablename = '?'
query = f'''
    create table if not exists {tablename} (
        id serial primary key,
        brand text,
        category text,
        sub_category text,
        product text,
        product_id text,
        product_sku text,
        img text,
        final_price text,
        regular_price text,
        disc_perc text,
        voucher text,
        voucher_price text,
        installment text,
        review text,
        rating text,
        product_link text,
        batch text
    );
'''
cur.execute(query)
conn.commit()

In [ ]:
def get_web(url):
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [ ]:
def crawl_insert(cat, j, articles, k, cur, conn, tablename):

    try:
        brand = articles[k]['data-brand'].strip().replace("'", "")
        category = cat[j][0]
        sub_category = articles[k]['data-category'].strip().replace("'", "")
        product = articles[k]['data-title'].strip().replace("'", "")
        product_id = articles[k]['data-seller-product-id'].strip()
        product_sku = articles[k]['product-sku'].strip()
        img = articles[k].img['src'].strip()
        final_price = articles[k].find_all('span', {'class': "final-price"})[0].text.strip().replace("đ", "").replace(".", "").split(' ')[0]
        regular_price = final_price if articles[k].find_all('span', {'class': "price-regular"}) == [] else articles[k].find_all('span', {'class': "price-regular"})[0].text.strip().replace("đ", "").replace(".", "").split(' ')[0]
        disc_perc = "0%" if articles[k].find_all('span', {'class': "sale-tag sale-tag-square"}) == [] else articles[k].find_all('span', {'class': "sale-tag sale-tag-square"})[0].text.strip().split(' ')[0]
        voucher = ["?" if articles[k].find_all('p', {'class': "price-tag"}) == [] else articles[k].find_all('span', {'class': "code"})[0].text][0]
        voucher_price = ["?" if articles[k].find_all('p', {'class': "price-tag"}) == [] else articles[k].find_all('span', {'class': "price"})[0].find('span').text.replace("đ", "").replace(".", "")][0]
        installment = ["?" if articles[k].find_all('span', {'class': "installment-price-v2"}) == [] else articles[k].find_all('span', {'class': "installment-price-v2"})[0].text][0]
        review = [articles[k].find_all('p', {'class': "review"})[0].text.strip('\(\)') if articles[k].find_all('p', {'class': "review"}) != [] else "Chưa có nhận xét"][0]
        rating = ["?" if articles[k].find_all('span', {'class': "rating-content"}) == [] else articles[k].find_all('span', {'class': "rating-content"})[0].find('span')['style'].split(':')[1]][0]
        product_link = 'https://tiki.vn' + articles[k].a['href']
        batch = strftime("%Y-%m-%d %H:%M:%S", localtime())

        query = f"""insert into {tablename} (brand, category, sub_category, product, product_id, product_sku, img, final_price, regular_price, disc_perc, voucher, voucher_price, installment, review, rating, product_link, batch)
            values('{brand}', '{category}', '{sub_category}', '{product}', '{product_id}', '{product_sku}', '{img}', '{final_price}', '{regular_price}', '{disc_perc}', 
                '{voucher}', '{voucher_price}', '{installment}', '{review}', '{rating}', '{product_link}', '{batch}'
            );
        """

        cur.execute(query)
        conn.commit()
        
    except Exception as err:
        print(err)

In [ ]:
soup = get_web('https://tiki.vn')
categories = soup.findAll('a', {'class', "MenuItem__MenuLink-sc-181aa19-1 fKvTQu"})
category, link = [], []

for h in range(len(categories)):
    try:
        link.append(categories[h]['href'])
        category.append(categories[h].text)
    except:
        print('pass')
        
cat = list(zip(category,link))

In [ ]:
for j in range(len(cat)):
    try:
        soup = get_web(cat[j][1])
        articles = soup.find_all('div', {'class': "product-item"})
        print('Looping: ' + cat[j][1])
        for k in range(len(articles)):
            crawl_insert(cat, j, articles, k, cur, conn, tablename)
  
        links = soup.find_all('div', {'class': "list-pager"})
        
        # Exit while loop when reaching to the last page
        while links[0].find_all('a', {'class': "next"}) != []:
            try:
                soup = get_web('https://tiki.vn' + links[0].find_all('a', {'class': "next"})[0]['href'])
                articles = soup.find_all('div', {'class': "product-item"})
                print('Looping: ', cat[j][0], links[0].find_all('a', {'class': "next"})[0]['href'].split('&')[1], sep=' ')
                for i in range(len(articles)):
                    crawl_insert(cat, j, articles, i, cur, conn, tablename)
                links = soup.find_all('div', {'class': "list-pager"})
            except:
                continue
    except:
        continue
        
print('Crawl done!')